<a href="https://colab.research.google.com/github/Jaseelkt007/ML/blob/master/Diabetic_Retinopathy_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
from multiprocessing import Pool
import warnings
from tqdm import tqdm
import os
import torch
import matplotlib.pyplot as plt
import cv2

sample_data_path = '/content/drive/MyDrive/sample/test_data'
output_folder = '/content/drive/MyDrive/sample/test_resized'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

#transform = transforms.Compose([
    #transforms.Resize((256,256)),
 #   transforms.RandomRotation(20),
  #  transforms.RandomHorizontalFlip(),
   # transforms.ColorJitter(brightness= 0.2, contrast = 0.2),
    #transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456,0.406], std= [0.229,0.224,0.225])
#])

def trim(image):

    percentage = 0.02
    img = np.array(image)
    img_gray = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY) # Convert to grayscale to simply the process
    # create the binary mask , to get the background from actual content
    img_gray = img_gray > 0.1 * np.mean(img_gray[img_gray!=0])
    # calculate the row wise and column wise sums to find where the significant content exists
    row_sums = np.sum(img_gray, axis = 1)
    col_sums = np.sum(img_gray, axis = 0)
    rows = np.where(row_sums > img.shape[1] * percentage)[0] # return the rows index of rows which contain atleast 2% of its content
    cols = np.where (col_sums > img.shape[0] * percentage)[0]
    # find the min and max rows and columns for croping
    min_row, min_col = np.min(rows), np.min(cols)
    max_row, max_col = np.max(rows), np.max(cols)
    im_crop = img[min_row : max_row +1 , min_col : max_col+1]
    return Image.fromarray(im_crop)

def resize_main_aspect(image, desired_size):
    old_size = image.size
    ratio = float(desired_size)/ max(old_size) # resize ratio
    new_size = tuple([int(x * ratio) for x in old_size]) # (N,M) N,M are new size
    im = image.resize(new_size, Image.LANCZOS) # a filter to smooth image when resize, helps to reduce artifacts in the reduced image
    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size - new_size[0])//2 , (desired_size - new_size[1])//2)) # paster the image on the new square background
    return new_im

def save_single(args): # helpfull for multiprocessing
    img_file, input_path_folder, output_path_folder, output_size = args
    image_org = Image.open(os.path.join(input_path_folder, img_file))
    image = trim(image_org)
    image = resize_main_aspect(image, desired_size= output_size[0])
    image.save(os.path.join(output_path_folder , img_file))



def multi_image_resize(input_path_folder, output_path_folder, output_size=None):
    if not output_size:
        warnings.warn("Need to specify output_size! For example: output_size=100")
        exit()

    if not os.path.exists(output_path_folder):
        os.makedirs(output_path_folder)

    jobs = [
        (file, input_path_folder, output_path_folder, output_size)
        for file in os.listdir(input_path_folder)
        if os.path.isfile(os.path.join(input_path_folder,file))
    ]

    with Pool() as p:
        list(tqdm(p.imap_unordered(save_single, jobs), total=len(jobs)))

#if __name__ == "__main__":
multi_image_resize(sample_data_path, output_folder, output_size = (256,256))


def preprocess_images(data_path, transform):
    processed_images = []
    for img_name in os.listdir(data_path):
        img_path = os.path.join(data_path, img_name)
        image = Image.open(img_path)
        image = trim(image)
        image_resized = resize_main_aspect(image, desired_size=256)
        image = transform(image_resized)
        processed_images.append(image)
    return processed_images

#processed_images = preprocess_images(sample_data_path, transform)

def show_images(images, n=5):
    fig, axs = plt.subplots(1, n , figsize=(15,5))
    for i , img in enumerate(images[:n]):
        img = img.permute(1,2,0) # change from C, H, W to H, W, C
        img = torch.clamp(img * torch.tensor([0.229,0.224,0.225]) +
                          torch.tensor([0.485,0.456,0.406]), 0,1) # denormalize
        axs[i].imshow(img)
        axs[i].axis("off")
    plt.show()

#show_images(processed_images,n=5)







100%|██████████| 103/103 [00:59<00:00,  1.72it/s]


In [3]:
import os
import csv
from PIL import Image



resized_folder = '/content/drive/MyDrive/sample/test_resized'
final_folder = '/content/drive/MyDrive/sample/test_resized/binary'
csv_file_path = '/content/test.csv'


if not os.path.exists(final_folder):
    os.makedirs(final_folder)

# Create output folder structure for each class
for i in range(2):
    class_folder = os.path.join(final_folder, f'class_{i}')
    os.makedirs(class_folder, exist_ok=True)

with open(csv_file_path, encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)
    image_name_index = header.index("Image name")
    grade_index = header.index("label")

    # Loop through each row in CSV and process images based on class
    for row in tqdm(csv_reader, desc='Processing images', unit='image'):
        image_name = row[image_name_index]
        label = row[grade_index]

        # Load the preprocessed image
        img_path = os.path.join(resized_folder, f"{image_name}.jpg")
        if not os.path.exists(img_path):
            print(f"Warning: {img_path} doesn't exist")
            continue  # Skip if the file doesn't exist

        image = Image.open(img_path)
        class_folder = os.path.join(final_folder, f'class_{label}')

        # Save the original image in the respective folder
        original_image_name = f"{image_name}.jpg"
        if not os.path.exists(os.path.join(class_folder, original_image_name)):  # Save only if it doesn't exist
            image.save(os.path.join(class_folder, original_image_name))


Processing images: 103image [00:01, 70.80image/s]


In [ ]:
# Minor augmentation to balance the classes
import os
import csv
import tensorflow as tf
from PIL import Image
from tqdm import tqdm

# Define augmentation functions for minor augmentation
def augment_image(image):  # Minor augmentation to balance the dataset
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomRotation(0.02),  # Small rotation
        tf.keras.layers.RandomBrightness(0.05),
        tf.keras.layers.RandomFlip("horizontal"),
    ])
    return data_augmentation(image)

# Paths and folders
preprocessed_folder = '/content/drive/MyDrive/sample/preprocessed_samples'
augmented_folder = '/content/drive/MyDrive/sample/aug_train_samples/binary'
csv_file_path = '/content/train.csv'

if not os.path.exists(augmented_folder):
    os.makedirs(augmented_folder)

# Create output folder structure for each class
for i in range(2):
    class_folder = os.path.join(augmented_folder, f'class_{i}')
    os.makedirs(class_folder, exist_ok=True)

# Set target augmentation limits (total count, including originals)
target_augmentation = {'0': 300, '1': 50}  # Total images needed per class
class_counters = {'0': 0, '1': 0}  # Track the count of images per class (original + augmented)

# Read the CSV file and process each image
with open(csv_file_path, encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)
    image_name_index = header.index("Image name")
    grade_index = header.index("label")

    # Loop through each row in CSV and process images based on class
    for row in tqdm(csv_reader, desc='Processing images', unit='image'):
        image_name = row[image_name_index]
        label = row[grade_index]
        label_str = str(label)

        # Load the preprocessed image
        img_path = os.path.join(preprocessed_folder, f"{image_name}.jpg")
        if not os.path.exists(img_path):
            print(f"Warning: {img_path} doesn't exist")
            continue  # Skip if the file doesn't exist

        image = Image.open(img_path)
        image_array = tf.keras.preprocessing.image.img_to_array(image)  # Convert PIL image to numpy array
        image_array = tf.image.convert_image_dtype(image_array, dtype=tf.float32)  # Scale pixel values to [0,1]

        # Define where to save images based on class label
        class_folder = os.path.join(augmented_folder, f'class_{label}')

        # Save the original image as `aug_0` for each file
        original_image_name = f"{image_name}_aug_0.jpg"
        if not os.path.exists(os.path.join(class_folder, original_image_name)):  # Save only if it doesn't exist
            image.save(os.path.join(class_folder, original_image_name))
            class_counters[label_str] += 1  # Count the augmented image

        # Generate one augmented image as `aug_1` only if the target count is not yet met
        if class_counters[label_str] < target_augmentation[label_str]:
            augmented_image = augment_image(image_array)
            augmented_image = tf.keras.preprocessing.image.array_to_img(augmented_image)  # Convert back to PIL image
            aug_image_name = f"{image_name}_aug_1.jpg"
            augmented_image.save(os.path.join(class_folder, aug_image_name))
            class_counters[label_str] += 1  # Count the augmented image

print("Original and augmented images saved with consistent naming.")

Processing images: 413image [00:17, 23.34image/s]

Original and augmented images saved with consistent naming.


In [ ]:
import tensorflow as tf
import logging

batch_size = 16

data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomRotation(factor=0.03),  # Approximately ±10 degrees,
        tf.keras.layers.RandomZoom(height_factor=0.1, width_factor=0.1), # small zoom
        tf.keras.layers.RandomBrightness(0.1),
        tf.keras.layers.RandomContrast(0.1),
        tf.keras.layers.RandomFlip("horizontal_and_vertical")
    ])

def augment(image, label):
    image = data_augmentation(image, training=True)
    return image, label

def preprocess(image, label, img_height=256, img_width=256):
    """Dataset preprocessing: Normalizing and resizing"""
    image = tf.image.resize(image, (img_height, img_width))
    # Normalize image to [0, 1] and resize
    image = tf.cast(image, tf.float32) / 255.0

    image = (image - [0.485, .456, 0.406])/ [0.229 , 0.224 ,0.225]

    return image, label



def load(name, data_dir, test_data_dir, batch_size= batch_size, caching=True):
    """Load datasets based on name"""
    if name == "idrid":
        logging.info(f"Preparing dataset {name}...")

        # Load dataset from directory structure, where each subdirectory represents a class,return an object, which is an iterable tuples (image, label)
        full_ds = tf.keras.preprocessing.image_dataset_from_directory(
            data_dir,
            batch_size=batch_size,
            label_mode='int' # use 'int' for integer label , for classification
        )

        # Calculate the number of examples for shuffle buffer size
        num_examples = len(full_ds) * batch_size
        for images, _ in full_ds.take(1):
            image_shape = images.shape[1:]
            break

        class_names = full_ds.class_names
        num_classes = len(class_names)

        # Define df_info
        ds_info = {
            "num_examples" : num_examples,
            "features" : {
                "image" : {"shape" : image_shape , "dtype": tf.float32},
                "label" : {"num_classes": num_classes, "dtype": tf.int64}
            }
        }

        # Split into training and validation sets
        val_size = int(0.2 * len(full_ds))
        train_size = len(full_ds) - val_size
        ds_train = full_ds.take(train_size)
        ds_val = full_ds.skip(train_size)

        # Load test data
        ds_test= None
        if test_data_dir:
            ds_test = tf.keras.preprocessing.image_dataset_from_directory(
                test_data_dir, batch_size = batch_size, label_mode = 'int'
            )

        # Prepare and return the training and validation datasets
        return prepare(ds_train, ds_val, ds_test = ds_test, ds_info = ds_info ,batch_size=batch_size, caching=caching)


def prepare(ds_train, ds_val, ds_test= None, ds_info=None, batch_size = batch_size , caching = True):
    """Prepare datasets with preprocessing, augmentation, batching, caching, and prefetching"""
    # Prepare training dataset
    ds_train = ds_train.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if caching:
        ds_train = ds_train.cache()
    if ds_info:
        shuffle_buffer_size = ds_info.get("num_examples", 1000) // 10  # Default to 1000 if ds_info not provided
        ds_train = ds_train.shuffle(shuffle_buffer_size)
    else:
        ds_train = ds_train.shuffle(1000)  # Fallback shuffle size
    ds_train = ds_train.repeat().prefetch(tf.data.AUTOTUNE)

    # Prepare validation dataset (no augmentation)
    ds_val = ds_val.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if caching:
        ds_val = ds_val.cache()
    ds_val = ds_val.prefetch(tf.data.AUTOTUNE)

    # Prepare test dataset if available (no augmentation)
    if ds_test is not None:
        ds_test = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        if caching:
            ds_test = ds_test.cache()
        ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

    return ds_train, ds_val, ds_test, ds_info




data_dir = '/content/drive/MyDrive/sample/aug_train_samples/binary'
test_data_dir = '/content/drive/MyDrive/sample/test_resized/binary'
ds_train , ds_val , _, _ = load("idrid" , data_dir, test_data_dir)


#for images, labels in ds_train.take(1):
 #   print("image batch shape: ", images.shape)
  #  print("Label batch shape :", labels.shape)

Found 588 files belonging to 2 classes.


In [ ]:
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, GlobalAveragePooling2D
# Load the pretrained Model VGG16
base_model = tf.keras.applications.VGG16(
    input_shape = (256,256,3),
    include_top = False,
    weights = "imagenet"
)

# Freeze the base model
base_model.trainable = False

# Build the model
model = tf.keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024),                # Dense layer without activation
    LeakyReLU(alpha=0.01),      # LeakyReLU with a small negative slope
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss = 'SparseCategoricalCrossentropy',
    metrics = ['accuracy']
)

small_ds = ds_train.take(10)


# Train the model
history = model.fit(small_ds, validation_data = ds_val , epochs=50 , verbose=1)

NameError: name 'tf' is not defined